In [47]:
import numpy as np
import pandas as pd
import math
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD 
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.feature_extraction import DictVectorizer
from xgboost.sklearn import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

In [48]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import time
start = time.time()

meta_file = pd.read_csv("./AutoKaggle - Metadata.csv")
arrOfRows = [64,360,239,316,515,518]
nlp_rows = [239]
tabular_rows = [64,360,316,515,518]
row = 518
competition_name = meta_file['name'].loc[row]
print(meta_file['name'].loc[row])
dir_name = "./" + competition_name 
train_file = dir_name + "/data/train.csv"
row_file = dir_name + "/submission/row.csv"

meta = pd.read_csv(row_file)
def preprocessing(row):
   
    train = pd.read_csv(train_file)
#     find_row = meta.loc[row]
#     train = ''
#     test = None
    check_pred = False
    train_X = ''
    train_Y = ''
    test_X = None
    if meta['name'].loc[0] == 'kobe-bryant-shot-selection':
        train = pd.read_csv("./kobe-bryant-shot-selection/data/data.csv")
        check_pred = False
    elif meta['name'].loc[0] == 'mercedes-benz-greener-manufacturing':
        train = pd.read_csv("./mercedes-benz-greener-manufacturing/data/train.csv")
        test = pd.read_csv("./mercedes-benz-greener-manufacturing/data/test.csv")
    elif meta['name'].loc[0] == 'uciml_sms-spam-collection-dataset':
        train = pd.read_csv("./uciml_sms-spam-collection-dataset/data/spam.csv", encoding='cp1252', error_bad_lines=False)
        check_pred = False
      
    
#     if check_pred:
#         test = test.dropna()
    if meta['name'].loc[0] == 'uciml_sms-spam-collection-dataset':
        row = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv", encoding='cp1252')
        sms = train
        row_prepro = row['preprocessing function call'][0]
        prepro_ls = eval(row_prepro)
        sms = eval(prepro_ls[0])
        train = eval(prepro_ls[1])
        return train
    else:
        if type(meta['preprocessing function call'].loc[0]) is str:
            preprocessing_calls = eval(meta['preprocessing function call'].loc[0])
            for call in preprocessing_calls:
                exec(call)
        train = train.dropna()
        if type(meta["unwanted column"].loc[0]) is str:  # check if there's unwanted column
            column_list = eval(meta["unwanted column"].loc[0])
            train.drop(column_list,axis=1)
        
        if type(meta["numeric column"].loc[0]) is str:
            numeric=eval(meta["numeric column"].loc[0])
        for c in train.columns:
            if train[c].dtype == 'object':    # deal with text
                lbl = LabelEncoder() 
                if check_pred:
                    lbl.fit(list(train[c].values) + list(test[c].values)) 
                    train[c] = lbl.transform(list(train[c].values))
                    test[c] = lbl.transform(list(test[c].values))
                else:
                    lbl.fit(list(train[c].values))
                    train[c] = lbl.transform(list(train[c].values))

        target_name = str(meta['targetName'].loc[0])
        print(target_name)
        train_Y = train[target_name]
        train_X = train.drop(columns=target_name)
        train_X,test_X,train_Y,test_Y = train_test_split(train_X, train_Y, test_size=0.2)
        if check_pred:
            pred_X = pred
            return train_X, train_Y,test_X,test_Y ,pred_X
        else:
            return train_X, train_Y, test_X,test_Y,None#         if check_pred:


preprocessing(row)


restaurant-revenue-prediction
revenue


(      Id  Open Date  City  City Group  Type  P1   P2   P3   P4  P5 ...    P28  \
 46    46         76    32           0     2   3  5.0  5.0  3.0   1 ...    3.0   
 53    53         28    26           1     1   3  4.0  4.0  3.0   1 ...    3.0   
 23    23         46    25           1     1   2  4.0  4.0  4.0   2 ...    2.0   
 18    18         33    32           0     2   4  5.0  4.0  3.0   1 ...    2.0   
 96    96        116    10           1     1   4  5.0  2.0  4.0   2 ...    2.0   
 74    74        119    31           0     2  12  7.5  6.0  6.0   2 ...   12.5   
 116  116         29    32           0     1   2  4.0  4.0  4.0   2 ...    3.0   
 4      4         41    14           1     2   3  4.0  3.0  4.0   2 ...    1.0   
 83    83         68    31           0     2   2  4.0  4.0  4.0   2 ...    2.0   
 72    72         62    32           0     2   3  5.0  4.0  4.0   2 ...    5.0   
 7      7         52    31           0     2   4  5.0  4.0  5.0   2 ...    3.0   
 81    81       

In [49]:
def text_process(text):
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    
    return " ".join(text)

def feature_extraction(row, X_train, X_test,X_pred):
    if meta['name'].loc[0] == 'uciml_sms-spam-collection-dataset':
        rowcsv = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv")
        row_extract = rowcsv['featureExtractor function call'].loc[0]
        sms = X_train
        extract = eval(row_extract)
        sms['message'] = eval(extract[0])
        sms['message'] = eval(extract[1])
        text_feat = sms['message'].apply(str).copy()
        text_feat = eval(extract[2])
        vectorizer = eval(extract[3])
        features = eval(extract[4])
        features_train, features_test, labels_train, labels_test = train_test_split(features, sms['label'], test_size=0.3)
        return features_train, features_test, labels_train, labels_test
    else:
        if type(meta["featureExtractor function call"].loc[0]) is not str:
            print('not func')
            return X_train,X_test,X_pred
        extraction_function_calls = str(row_data["function call feature extraction"].loc[0])
        extraction_function_calls = extraction_function_calls.split(",")
        extraction_funtion_param = eval(row_data["function parameters feature extraction"].loc[0])
        function_nums = len(extraction_function_calls)
        for i in range(function_nums):
            str1 = extraction_function_calls[i]
            str2 = extraction_funtion_param[i]
            l_str = str1.split("(")
            l_str.insert(1,"("+str2)
            str_call = ''
            str_call = str_call.join(l_str)
            str_call = 'extractor' + '=' + str_call
            exec(str_call, globals(), globals())
            extracted_train = extractor.fit_transform(X_train)
            n_comp = extracted_train.shape[1]
            for j in range(0, n_comp):
                X_train['extractor'+ str(i)+"_"+str(j)] = extracted_train[:, j]
            if X_test is not None:
                extracted_test = extractor.fit_transform(X_test)
                for j in range(0, n_comp):
                    X_test['extractor'+ str(i)+"_"+str(j)] = extracted_test[:, j]
                return X_train,X_test
            else:
                return X_train,None

In [50]:
def feature_selection():
    pass

In [51]:
def estimation(row,X_train,X_test,Y_train, Y_test):
    if row in nlp_rows:
        rowcsv = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv")
        row_extract = eval(rowcsv['estimator1 function call'].loc[0])
        mnb = eval(row_extract[0])
        eval(row_extract[1])
        pred = eval(row_extract[2])
        if rowcsv['performanceMetric'].loc[0] == 'accuracy':
            return accuracy_score(Y_test, pred)
    else:
        estimation_function_calls = eval(meta["estimator1 function call"].loc[0])
        print(len(estimation_function_calls))
        if len(estimation_function_calls) == 1:
            str_call = estimation_function_calls[0]
            print(str_call)
            str_call = 'estimator' + '=' + str_call
            print(str_call)
            exec(str_call,globals(),globals())
            
            if meta["taskType"].loc[0] == 'classification':
                estimator.fit(X_train,Y_train)
                Y_pred = estimator.predict(X_test)
                print('here')
                print(accuracy_score(Y_test,Y_pred))
                print('here')
            elif meta["taskType"].loc[0] == 'regression':
                estimator.fit(X_train,Y_train)
                Y_pred = estimator.predict(X_test)
                print(mean_squared_error(Y_test,Y_pred))
        else:
            estimators = []
            n_estimators = len(estimation_function_calls)
            for i in range(n_estimators):
                str1 = extraction_function_calls
                str2 = extraction_funtion_param
                l_str = str1.split("(")
                l_str.insert(1,"("+str2)
                str_call = ''
                str_call = str_call.join(l_str)
                str_call = 'estimator' + '=' + str_call
                print(l_str)
                print(str_call)
                exec(str_call)
                estimators.append(estimator)
                postprocessing(estimators,stack = True)

#                 estimators.append(estimator)
#                 postprocessing(estimators,stack = True)

In [52]:
if row in nlp_rows:
    train_set = preprocessing(row)
    X_train, X_test, Y_train, Y_test = feature_extraction(row, train_set, None)
    Y_pred = estimation(row, X_train, X_test, Y_train, Y_test)
    print(Y_pred)
if row in tabular_rows:
    X_train,Y_train,X_test,Y_test,X_pred = preprocessing(row)
    X_train,X_test,X_pred = feature_extraction(row,X_train,X_test,X_pred)
    estimation(row,X_train,X_test,Y_train, Y_test)

revenue
not func


AttributeError: 'RandomForestRegressor' object has no attribute 'estimators_'

In [53]:
# if meta['name'].loc[row] == 'uciml_sms-spam-collection-dataset':
#     train_set = preprocessing(row)
#     X_train, X_test, Y_train, Y_test = feature_extraction(row, train_set, None)
#     Y_pred = estimation(row, X_train, X_test, Y_train, Y_test)
#     print(Y_pred)

In [54]:
def postprocessing(estimators,stack):
    pass

In [55]:
end = time.time()
print("Use", end - start, "seconds to run this.")

Use 0.8779239654541016 seconds to run this.
